<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - Bias MRR</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [2]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender
from lenskit.algorithms.basic import Bias
from lenskit import topn
import pandas as pd
import numpy as np

ratings = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 1M\\ratings.dat', sep='::',engine='python',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [3]:
def model_trainer(item,user,damping):
    all_recs = []
    test_data = []
    
    bias = Bias(items=item, users=user, damping=damping) # define algorithm
    
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('Bias', bias, train, test))
    
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla = topn.RecListAnalysis()
    rla.add_metric(topn.recip_rank)
    results = rla.compute(all_recs, test_data)
                          
    return results["recip_rank"].mean()

In [4]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    user = params['user']
    item = params['item']
    damping = params['damping']
    metric = model_trainer(item,user,damping)
    print([item,user,damping,metric],',')
    return {'loss': -metric, 'status': STATUS_OK }

space={'item': hp.choice('item', ['True', 'False']),
       'user': hp.choice('user', ['True', 'False']),
       'damping': hp.uniform('damping', 0, 1)}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

['True', 'False', 0.5468391029595883, 0.04643085388316454]                                                             
,                                                                                                                      
['False', 'False', 0.7618040530385572, 0.05369265464661255]                                                            
,                                                                                                                      
['True', 'False', 0.0491430644983053, 0.028923524809250502]                                                            
,                                                                                                                      
['False', 'False', 0.4886088253241233, 0.03888215023050736]                                                            
,                                                                                                                      
['False', 'True', 0.1801061848989799, 0.

['False', 'False', 0.6842791200935343, 0.05334918603424073]                                                            
,                                                                                                                      
['False', 'False', 0.9282413801800933, 0.05483227033267059]                                                            
,                                                                                                                      
['False', 'False', 0.3117305566404489, 0.03136732452257102]                                                            
,                                                                                                                      
['False', 'False', 0.7914669423035516, 0.054321001150746395]                                                           
,                                                                                                                      
['False', 'False', 0.8838215891455872, 0

['False', 'True', 0.9397351855708428, 0.05815784973809309]                                                             
,                                                                                                                      
['False', 'True', 0.8046214553373376, 0.05649662399610745]                                                             
,                                                                                                                      
['False', 'True', 0.7576602440574379, 0.05384932568660376]                                                             
,                                                                                                                      
['False', 'True', 0.8344695965024966, 0.0561564735727335]                                                              
,                                                                                                                      
['False', 'True', 0.948585783471894, 0.0

In [5]:
array = [['True', 'False', 0.5468391029595883, 0.04643085388316454]              #34.71                                               
,                                                                                                                      
['False', 'False', 0.7618040530385572, 0.05369265464661255]                                                            
,                                                                                                                      
['True', 'False', 0.0491430644983053, 0.028923524809250502]                                                            
,                                                                                                                      
['False', 'False', 0.4886088253241233, 0.03888215023050736]                                                            
,                                                                                                                      
['False', 'True', 0.1801061848989799, 0.03013771249491572]                                                             
,                                                                                                                      
['True', 'True', 0.379241056304365, 0.03220785060162322]                                                               
,                                                                                                                      
['True', 'False', 0.005942658255959632, 0.02824631006489465]                                                           
,                                                                                                                      
['False', 'False', 0.8797885468170499, 0.055445422928121484]                                                           
,                                                                                                                      
['False', 'True', 0.4181229661820851, 0.03005820468907594]                                                             
,                                                                                                                      
['False', 'True', 0.2059158137688717, 0.02949053801337157]                                                             
,                                                                                                                      
['False', 'True', 0.07737334171461674, 0.02919072202637387]                                                            
,                                                                                                                      
['True', 'True', 0.6125260512904598, 0.050204528545772564]                                                             
,                                                                                                                      
['True', 'True', 0.5960461972857958, 0.04917584396751812]                                                              
,                                                                                                                      
['False', 'True', 0.9202214711548267, 0.05523138061889318]                                                             
,                                                                                                                      
['True', 'False', 0.4577953201835654, 0.035180901903513855]                                                            
,                                                                                                                      
['True', 'False', 0.17021035971827148, 0.03013414665899075]                                                            
,                                                                                                                      
['True', 'False', 0.44865981660120324, 0.032009834922610156]                                                           
,                                                                                                                      
['True', 'True', 0.4035020922820285, 0.03153368100782981]                                                              
,                                                                                                                      
['False', 'True', 0.856536355290169, 0.05450766960887118]                                                              
,                                                                                                                      
['False', 'True', 0.17156122901037107, 0.0299346294778612]                                                             
,                                                                                                                      
['False', 'False', 0.998317040622877, 0.05802185551078529]                                                             
,                                                                                                                      
['False', 'False', 0.9843403489452248, 0.05844306230997172]                                                            
,                                                                                                                      
['False', 'False', 0.9984817701341357, 0.056968124275982975]                                                           
,                                                                                                                      
['False', 'False', 0.731235741655865, 0.05356906042468878]                                                             
,                                                                                                                      
['False', 'False', 0.9946392532024818, 0.05928885716077152]                                                            
,                                                                                                                      
['False', 'False', 0.7481175787537607, 0.053124024960634744]                                                           
,                                                                                                                      
['False', 'False', 0.9855090915415602, 0.06032079955885739]                                                            
,                                                                                                                      
['False', 'False', 0.8268593946361634, 0.05516318373991216]                                                            
,                                                                                                                      
['False', 'False', 0.657771641475642, 0.05179074537430173]                                                             
,                                                                                                                      
['False', 'False', 0.3082071515349867, 0.03130414486407523]                                                            
,                                                                                                                      
['False', 'False', 0.9392743883125755, 0.057079562278460595]                                                           
,                                                                                                                      
['False', 'False', 0.8326661588281492, 0.05526655327692941]                                                            
,                                                                                                                      
['False', 'False', 0.7858669662510547, 0.05328305157217619]                                                            
,                                                                                                                      
['False', 'False', 0.528563460764253, 0.04701205524251689]                                                             
,                                                                                                                      
['False', 'False', 0.6842791200935343, 0.05334918603424073]                                                            
,                                                                                                                      
['False', 'False', 0.9282413801800933, 0.05483227033267059]                                                            
,                                                                                                                      
['False', 'False', 0.3117305566404489, 0.03136732452257102]                                                            
,                                                                                                                      
['False', 'False', 0.7914669423035516, 0.054321001150746395]                                                           
,                                                                                                                      
['False', 'False', 0.8838215891455872, 0.05392143710933568]                                                            
,                                                                                                                      
['True', 'False', 0.6753568620047767, 0.05215370849317674]                                                             
,                                                                                                                      
['False', 'False', 0.9626683854182108, 0.057674296403006066]                                                           
,                                                                                                                      
['False', 'False', 0.5694477245115535, 0.04808555312385046]                                                            
,                                                                                                                      
['True', 'False', 0.6330848417456185, 0.050718497264150154]                                                            
,                                                                                                                      
['False', 'False', 0.7113732439781988, 0.05388230660805614]                                                            
,                                                                                                                      
['False', 'True', 0.8809990271485821, 0.055914372091188165]                                                            
,                                                                                                                      
['True', 'False', 0.5136779993328259, 0.042643097688226925]                                                            
,                                                                                                                      
['False', 'True', 0.03571490677717104, 0.02792394037410137]                                                            
,                                                                                                                      
['True', 'False', 0.32793604897304185, 0.03171315503149988]                                                            
,                                                                                                                      
['False', 'True', 0.9048202898108876, 0.05618175644088867]                                                             
,                                                                                                                      
['False', 'False', 0.09644262141667292, 0.02936775029079132]                                                           
,                                                                                                                      
['True', 'False', 0.8111345595111625, 0.05454789954382513]                                                             
,                                                                                                                      
['False', 'True', 0.7694641215688559, 0.05214579003040912]                                                             
,                                                                                                                      
['False', 'False', 0.5657787696991624, 0.04790456654877498]                                                            
,                                                                                                                      
['True', 'False', 0.22157767186606125, 0.02913634266300727]                                                            
,                                                                                                                      
['False', 'True', 0.9643737642098276, 0.058748400419771744]                                                            
,                                                                                                                      
['False', 'False', 0.8565062197224913, 0.05761503758225095]                                                            
,                                                                                                                      
['False', 'False', 0.709548653872589, 0.05296633681983201]                                                             
,                                                                                                                      
['True', 'True', 0.4684609329525906, 0.038236271817018386]                                                             
,                                                                                                                      
['False', 'False', 0.6155113975295746, 0.050569507491880474]                                                           
,                                                                                                                      
['False', 'False', 0.3619543996836849, 0.03096211879561776]                                                            
,                                                                                                                      
['False', 'False', 0.2335995979162011, 0.029966054423287]                                                              
,                                                                                                                      
['False', 'True', 0.9900620293935476, 0.05665214207918771]                                                             
,                                                                                                                      
['True', 'False', 0.1174701674298187, 0.028941254445749378]                                                            
,                                                                                                                      
['False', 'False', 0.8550566553848759, 0.05594814649144695]                                                            
,                                                                                                                      
['False', 'False', 0.4247689296644382, 0.03188358007701789]                                                            
,                                                                                                                      
['False', 'True', 0.9722938189746698, 0.059052552229427326]                                                            
,                                                                                                                      
['False', 'True', 0.9598388989548282, 0.059812838179173114]                                                            
,                                                                                                                      
['False', 'True', 0.9156989949906256, 0.05622799824438467]                                                             
,                                                                                                                      
['False', 'True', 0.9397351855708428, 0.05815784973809309]                                                             
,                                                                                                                      
['False', 'True', 0.8046214553373376, 0.05649662399610745]                                                             
,                                                                                                                      
['False', 'True', 0.7576602440574379, 0.05384932568660376]                                                             
,                                                                                                                      
['False', 'True', 0.8344695965024966, 0.0561564735727335]                                                              
,                                                                                                                      
['False', 'True', 0.948585783471894, 0.05688455758135398]                                                              
,                                                                                                                      
['False', 'True', 0.8929733370001119, 0.05602846363438921]                                                             
,                                                                                                                      
['False', 'True', 0.999330366676471, 0.0581152464547114]                                                               
,                                                                                                                      
['False', 'True', 0.7355176114175951, 0.05455831501742708]                                                             
,                                                                                                                      
['False', 'False', 0.8606425903196674, 0.05561405973493684]                                                            
,                                                                                                                      
['False', 'True', 0.777103101205413, 0.053864771743858696]                                                             
,                                                                                                                      
['True', 'False', 0.9216344037321309, 0.05930044984709632]                                                             
,                                                                                                                      
['True', 'False', 0.6456950863516745, 0.05239146309101524]                                                             
,                                                                                                                      
['True', 'True', 0.7179161865647128, 0.05318113261188619]                                                              
,                                                                                                                      
['True', 'False', 0.9156661517735154, 0.05780816003057771]                                                             
,                                                                                                                      
['True', 'False', 0.677314630105476, 0.053368534486533996]                                                             
,                                                                                                                      
['True', 'True', 0.8176559581114157, 0.05283130582739999]                                                              
,                                                                                                                      
['True', 'False', 0.9563866285079269, 0.06093895201104277]                                                             
,                                                                                                                      
['True', 'False', 0.9742358972139074, 0.05784037022818539]                                                             
,                                                                                                                      
['True', 'True', 0.9997181948960039, 0.05919180597216512]                                                              
,                                                                                                                      
['True', 'False', 0.49263051071235664, 0.038427638184583814]                                                           
,                                                                                                                      
['True', 'False', 0.9537616373836597, 0.05866636516351199]                                                             
,                                                                                                                      
['True', 'True', 0.253555604052067, 0.030517583079741183]                                                              
,                                                                                                                      
['True', 'False', 0.5742504228249264, 0.047326836372697516]                                                            
,                                                                                                                      
['True', 'False', 0.8734546157303902, 0.054282465110404084]                                                            
,                                                                                                                      
['True', 'True', 0.8389878422653304, 0.05237792663753947]                                                              
,                                                                                                                      
['True', 'False', 0.5378652353667375, 0.04695302076407894]                                                             
,                                                                                                                      
['True', 'False', 0.8927564708545246, 0.0556793623620764]                                                              
,                                                                                                                      
['False', 'True', 0.8062647567773187, 0.055466191814830076]                                                            
,                                                                                                                      
['True', 'False', 0.6993556516041972, 0.05107005555023186]                                                             
,                                                                                                                      
['False', 'False', 0.6054448902737726, 0.05038875549136568]                                                            
,                                                                                                                      
['False', 'True', 0.7489255349552026, 0.054225363613822575]                                                            
,                                                                                                                      
['True', 'False', 0.9403557935698741, 0.05857543781963407]]

In [6]:
metric = pd.DataFrame(array)
metric.columns = ['items','users','damping','MRR']
metric.head(5)

,items,users,damping,MRR
0,True,False,0.546839,0.046431
1,False,False,0.761804,0.053693
2,True,False,0.049143,0.028924
3,False,False,0.488609,0.038882
4,False,True,0.180106,0.030138


In [7]:
metric.sort_values(by=['MRR'],ascending=False).head(10)

,items,users,damping,MRR
84,True,False,0.956387,0.060939
26,False,False,0.985509,0.060321
66,False,True,0.959839,0.059813
78,True,False,0.921634,0.059300
24,False,False,0.994639,0.059289
86,True,True,0.999718,0.059192
65,False,True,0.972294,0.059053
54,False,True,0.964374,0.058748
88,True,False,0.953762,0.058666
99,True,False,0.940356,0.058575


In [8]:
metric.to_csv('metric_mrr.csv', index=False)